## Lab 7: File Compression

### Simulating the data:

Using np.random.choice, generate 100 megabytes(8 bits/byte* 1024 bytes/kilobyte* 1024 kilobytes/megabyte* 100)of random data containing 100%, 90%, 80%, 70%, 60%, and 50% zeros.

In [1]:
import pandas as pd
import numpy as np

In [2]:
def generate_file(p1):
    myvar = np.random.choice([0, 1], size=838860800, replace=True, p=[p1, 1-p1])
    myvar = np.packbits(myvar)
    return myvar

In [3]:
myvar_50 = generate_file(0.5)
myvar_60 = generate_file(0.6)
myvar_70 = generate_file(0.7)
myvar_80 = generate_file(0.8)
myvar_90 = generate_file(0.9)
myvar_100 = generate_file(1.0)

In [4]:
open('zeros_100p', 'wb').write(myvar_100)
open('zeros_90p', 'wb').write(myvar_90)
open('zeros_80p', 'wb').write(myvar_80)
open('zeros_70p', 'wb').write(myvar_70)
open('zeros_60p', 'wb').write(myvar_60)
open('zeros_50p', 'wb').write(myvar_50)

104857600

**Generate DNA sequence 100 million letters long:**

In [5]:
mydna = np.random.choice(['A','C','G','T'], size=100000000, replace=True, p=[0.25, 0.25, 0.25, 0.25])

In [6]:
open('mydna.fa', 'w').write(''.join(mydna))

100000000

### Compressing the data:  

Run on terminal via ssh. On each of the files I generated above, run gzip, bzip, pbzip2 and ArithmeticCompress.  

```time gzip -k zeros_100p```  
real	0m0.729s  
user	0m0.662s  
sys	0m0.064s  

```time bzip2 -k zeros_100p```  
real	0m1.021s  
user	0m0.977s  
sys	0m0.044s  

```time pbzip2 -k zeros_100p```  
real	0m0.105s  
user	0m1.798s  
sys	0m0.079s  

```time ArithmeticCompress zeros_100p zeros_100p.art```  
real	0m14.810s  
user	0m14.761s  
sys	0m0.049s  

In [9]:
from pandas import read_csv

df = pd.read_csv('compression_table.csv')
df

,filename,gzip_real,gzip_user,gzip_sys,bzip2_real,bzip2_user,bzip2_sys,pbzip2_real,pbzip2_user,pbzip2_sys,ac_real,ac_user,ac_sys
0,zeros_50p,3.514,3.398,0.112,16.651,16.559,0.092,1.479,38.992,0.937,40.814,40.648,0.104
1,zeros_60p,4.379,4.231,0.144,16.500,16.416,0.084,1.309,35.775,0.778,40.835,40.699,0.136
2,zeros_70p,6.009,5.933,0.072,14.493,14.385,0.108,1.172,30.203,0.760,39.650,39.278,0.336
3,zeros_80p,13.392,13.189,0.193,12.511,12.395,0.116,0.928,23.208,0.743,35.742,35.364,0.328
4,zeros_90p,19.114,19.012,0.092,11.626,11.553,0.072,0.758,18.908,0.847,29.583,29.354,0.228
5,zeros_100p,0.729,0.662,0.064,1.021,0.977,0.044,0.105,1.798,0.079,14.810,14.761,0.049
6,mydna,12.161,12.068,0.092,9.463,9.422,0.040,0.678,16.474,0.696,21.248,21.180,0.068


In [10]:
df2 = pd.read_csv('size_table.csv')
df2

,filename,initial_size,gzip,bzip2,pbzip2,ac
0,zeros_100p,105MB,102KB,113B,5.62KB,1.03KB
1,zeros_90p,105MB,58.7MB,61.2MB,61.2MB,49.2MB
2,zeros_80p,105MB,81.2MB,86.8MB,86.7MB,75.7MB
3,zeros_70p,105MB,93.6MB,99.8MB,99.8MB,92.4MB
4,zeros_60p,105MB,102MB,105MB,105MB,102MB
5,zeros_50p,105MB,105MB,105MB,105MB,105MB
6,mydna,100MB,29.3MB,27.3MB,27.3MB,25MB


**Which algorithm achieves the best level of compression on each file type?**  
ArithmeticCompress  

**Which algorithm is the fastest?**  
gzip  

**What is the difference between bzip2 and pbzip2? Do you expect one to be faster and why?**  
pbzip2 uses multiple threads and achieves near-linear speedup on SMP machines. pzip2 is slower than bzip2.  

**How does the level of compression change as the percentage of zeros increases? Why does this happen?**  
When number of zeros increases, the level of compression increases significantly. The more number of zeros appears, the easier it is to predict.  

**What is the minimum number of bits required to store a single DNA base?**  
One nucleotide can be stored by 2 bits (00, 01, 10, 11).  
For FASTA file:  (2 bits/nu) + (8 bits for <) + (8 bits for single-letter name) + (8 bits for \n) = 26 bits  

**What is the minimum number of bits requiredto store an amino acid letter?**  
One amino acid can be stored by 8 bits.  
For FASTA file: (8 bits/aa) + (8 bits for <) + (8 bits for single-letter name) + (8 bits for \n) = 32 bits  

**In your tests, how many bits did gzip and bzip2 actually require to store your random DNA and protein sequences?** gzip: 29.3MB = 29.3 x 1024 x 1024 x 8 = 245786214.4 bits  
bzip2: 27.3MB = 27.3 x 1024 x 1024 x 8 = 229008998.4 bits  

**Are gzip and bzip2 performing well on DNA and proteins?**  
Yes but ArithmeticCompress can do better.  

### Compressing real data:

**A priori, do you expect to achieve better or worse compression here than random data? Why?**  
I expect to achieve better compression than random data because number of repeated sequences usually higher in real sequence than in random data.

### Estimating compression of 1000 terabytes:  
800 TB Re-sequencing of genomes and plasmids: use ArithmeticCompress  
100 TB Protein sequences:  
100 TB Binary microscope images:  